In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/player-stats/all-teams/45?conference=12&division=-1&playertype=skater&position=skaters&rookie=no&sort=points&statstype=standard&page=1&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for i, row in enumerate(rows[1:], start=1):  
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for i, ele in enumerate(cols) if i != 0 and i != 2 and i != len(cols) - 1]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Jersey#', 'Name', 'Pos', 'Division', 'Team', 'GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'SHG', 'GWG', 'SOGW', 'PIM', 'SH%']

df = pd.DataFrame(acha_dynamic_stats, columns=column_names)

#Skip every other row, keeping only odd-indexed rows 
df = df.iloc[::2]
df = df[df['GP'].notna()]

#Handle Jersey#, SHG, and PIM to keep NA values as NA
for col in ['Jersey#', 'SHG', 'PIM']:
    df[col] = df[col].apply(lambda x: int(float(x)) if x else pd.NA)

#Reset the index to re-index the rows
df = df.reset_index(drop=True)

#Convert specified columns to Int64 to handle nullable integers
for col in ['Jersey#', 'SHG', 'PIM']:
    df[col] = df[col].astype('Int64')

numeric_columns = ['GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'GWG', 'SOGW', 'SH%']
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')  

string_columns = ['Name', 'Pos', 'Division', 'Team']
for column in string_columns:
    df[column] = df[column].astype(str)

print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_wd2_skaters.csv', index=False)

#Null values reflect website

      Jersey#                Name Pos         Division  \
0          16         Ellie Schau   F          WD2 CHE   
1          24    Nicole Partridge   F        WD2 IWCHL   
2          24        Sam Sitterly   F  WD2 Independent   
3          91   Sophia Buckberger   F        WD2 CCWHA   
4           7       Camryn Browne   F        WD2 DVCHC   
...       ...                 ...  ..              ...   
1423        7    Shannon O'Conner   F        WD2 CCWHA   
1424       30         Rachel Hart   G        WD2 CCWHA   
1425       21        Kayla Obrien   D          WD2 CHE   
1426       31    Samantha Steciak   G        WD2 CCWHA   
1427       43  Sophia Kaplunovich   D        WD2 DVCHC   

                                       Team  GP   G   A  PTS  Pt/G  PPG  SHG  \
0                 WD2 Mercyhurst University  13  29  12   41  3.15    1    3   
1               WD2 Sacred Heart University  12  24  12   36  3.00    7    0   
2           WD2 Eastern Michigan University  10  17  17   34  3